# Análise das Intents

Este jupyter-notebook vai auxiliar na análise de detecção de intenções de seu chatbot.

### Configurando e Imports

In [1]:
from IPython.display import IFrame

import rasa_nlu
print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.15.0


### Treinando o modelo do Rasa NLU

* Para avaliar o bot o primeiro passo é treiner o seu chatbot. Mas não é necessário treinar a parte de conversão completa (rasa_core) apenas a parte de interpretação de mensagens (rasa_nlu).

* O comando `train-nlu` do Makefile executa o treinamento apenas do `rasa_nlu`

In [2]:
!make train-nlu -C $COACH_DIR_PATH

make: Entering directory '/work/coach'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
--data data/intents/ -o /src_models --project nlu --verbose
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-07-17 16:30:02 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/geral.md is md
2019-07-17 16:30:02 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'elogios', 'negar', 'despedir', 'diga_mais', 'tudo_bem', 'out_of_scope', 'cumprimentar'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

2019-07-17 16:30:02 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/aleatorio.md is md
2019-07-17 16:30:02 INFO     rasa_nlu.traini

### Métodos de avaliação do chatbot

* O Rasa fornece vários métodos de avaliação e validação das `intents`, para verificar como utiliza-los, cada método fornece um log, imagem, gráfico ou arquivo com dados relevantes para interpretação do chatbot.

* O comando `!python -m rasa_nlu.test` é a base para a avaliação do chatbot.

* Na célula abaixo a flag `-h` foi utilizada para mostrar as funções e a forma de uso de cada uma delas, mude seus valores e flags para ter as informações desejadas na sua análise.

* Atualmente o Rasa possui 2 modos, `evaluation` e `crossvalidation` que tem seções decicadas a eles neste jupyter notebook.

In [3]:
!python -m rasa_nlu.test -h

/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: test.py [-h] [--debug] [-v] -d DATA [--mode MODE] [-c CONFIG]
               [-m MODEL] [-f FOLDS] [--report [REPORT]]
               [--successes [SUCCESSES]] [--errors ERRORS]
               [--histogram HISTOGRAM] [--confmat CONFMAT]

evaluate a Rasa NLU pipeline with cross validation or on external data

optional arguments:
  -h, --help            show this help message and exit
  --debug               Print lots of debugging statements. Sets logging level
                        to DEBUG
  -v, --verbose         Be verbose. Sets logging level to INFO
  -d DATA, --data DATA  file containing training/evaluation data
  --mode MODE           evaluation|crossvalidation (evaluate pretrained model
                        or train model by cross

### Evaluation

* O comando abaixo gera informações relevates para a validação das `intents` são elas:
    * Matriz de confusão
    * Histograma
    * Erros de intents

In [4]:
!python -m rasa_nlu.test -d $COACH_INTENTS_PATH -m $COACH_MODELS_NLU_PATH --mode evaluation

/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-07-17 16:31:33.800517: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-17 16:31:33.828980: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2194805000 Hz
2019-07-17 16:31:33.829499: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55c71b5187e0 executing computations on platform Host. Devices:
2019-07-17 16:31:33.829554: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-07-17 16:31:34 WARNING  tensorflow  - From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.ch

2019-07-17 16:31:44 INFO     __main__  - F1-Score:  1.0
2019-07-17 16:31:44 INFO     __main__  - Precision: 1.0
2019-07-17 16:31:44 INFO     __main__  - Accuracy:  1.0
2019-07-17 16:31:44 INFO     __main__  - Classification report: 
              precision    recall  f1-score   support

         bff       1.00      1.00      1.00        17
      comida       1.00      1.00      1.00        17
         cor       1.00      1.00      1.00        13
     esporte       1.00      1.00      1.00         6
      filhos       1.00      1.00      1.00        11
       filme       1.00      1.00      1.00         4
      genero       1.00      1.00      1.00        15
    historia       1.00      1.00      1.00         6
       hobby       1.00      1.00      1.00        11
         how       1.00      1.00      1.00        24
     license       1.00      1.00      1.00        14
  linguagens       1.00      1.00      1.00        14
        live       1.00      1.00      1.00        13
          

#### Matriz de Confusão
* A matriz de confusão mostra a correlação entre as intents.
* A diagonal principal tem forte correlação pois mostra a relação de uma intent **com ela mesma**
* O ideal é que não haja **nenhum valor** diferente de **0 fora da diagonal principal**.

In [5]:
IFrame(src='./confmat.png', width=900, height=700)

#### Erros
* O arquivo `erros,json` mostra os erros encontrados. Este arquivo lista os mesmos erros mostrados na **matriz de confusão**, então caso ela fique muito grande, não se preocupe, pois você pode apenas procurar os erros no arquivo gerado.
* Os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.
* Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [6]:
%cat errors.json

[
    {
        "text": "como vai",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.7145657539367676
        }
    },
    {
        "text": "tudo bom",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.7155274152755737
        }
    },
    {
        "text": "e o c#",
        "intent": "linguagens",
        "intent_prediction": {
            "name": "",
            "confidence": 0.0
        }
    }
]

#### Histograma

* O histograma contém a distribuição da predições das `intents`

In [7]:
IFrame(src='./hist.png', width=900, height=700)

### Crossvalidation

In [8]:
!python -m rasa_nlu.test -d $COACH_INTENTS_PATH -c $COACH_NLU_CONFIG_PATH --mode crossvalidation --folds 2 --report

/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-07-17 16:32:19 INFO     rasa_nlu.training_data.loading  - Training data format of ../../coach/data/intents/geral.md is md
2019-07-17 16:32:19 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'cumprimentar', 'tudo_bem', 'negar', 'elogios', 'despedir', 'out_of_scope', 'diga_mais'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

2019-07-17 16:32:19 INFO     rasa_nlu.training_data.loading  - Training data format of ../../coach/data/intents/aleatorio.md is md
2019-07-17 16:32:19 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (24 distinct intents)
	- Found intents: 'bff', 'risada', 'filhos', 'genero', 'p

2019-07-17 16:32:22.678227: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-17 16:32:22.700933: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2194805000 Hz
2019-07-17 16:32:22.701522: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55be5a7c66c0 executing computations on platform Host. Devices:
2019-07-17 16:32:22.701579: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-07-17 16:32:22 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:05<00:00, 53.69it/s, loss=0.194, acc=0.992]
2019-07-17 16:32:28 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Finished training embedding classifier, loss=0.194, train accuracy=0.992
2019-07-17 16:32:28 INFO     rasa_nlu.model  - Finished training compone

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `intents` são:

* [Evaluation](https://rasa.com/docs/nlu/evaluation/)